## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-17-11-28 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/21/us/sup...
1,2026-01-21-17-11-02 +0000,nypost,Ringleader of $250M Minnesota welfare fraud sc...,https://nypost.com/2026/01/21/us-news/minnesot...
2,2026-01-21-17-06-49 +0000,nyt,Trump Appears to Confuse Iceland and Greenland,https://www.nytimes.com/2026/01/21/us/politics...
3,2026-01-21-17-04-08 +0000,nypost,Manhattan judge rejects House lawmakers’ bid f...,https://nypost.com/2026/01/21/us-news/manhatta...
4,2026-01-21-17-03-57 +0000,nypost,Wild video shows Dem lawmakers get in vicious ...,https://nypost.com/2026/01/21/us-news/wild-vid...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,95
15,greenland,40
76,new,18
95,davos,17
388,court,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,192
17,2026-01-21-16-41-18 +0000,nyt,Trump Demands Control of Greenland but Rules O...,https://www.nytimes.com/2026/01/21/us/politics...,189
9,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...,183
250,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,173
222,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,171


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,192,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...
272,68,2026-01-20-23-45-18 +0000,latimes,State GOP seeks Supreme Court injunction to bl...,https://www.latimes.com/california/story/2026-...
247,61,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
9,46,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...
64,44,2026-01-21-15-22-38 +0000,nyt,Man Gets Life in Prison for Killing Shinzo Abe...,https://www.nytimes.com/2026/01/20/world/asia/...
177,44,2026-01-21-09-55-53 +0000,bbc,Train driver killed in second deadly Spain rai...,https://www.bbc.com/news/articles/c1m78xl0gmpo...
158,44,2026-01-21-10-51-01 +0000,bbc,UK inflation rises for first time in five mont...,https://www.bbc.com/news/articles/cnvg251g25yo...
205,38,2026-01-21-06-23-25 +0000,nypost,Iran’s top diplomat vows to ‘fire back with ev...,https://nypost.com/2026/01/21/world-news/irans...
199,37,2026-01-21-07-22-15 +0000,nypost,Israel’s Netanyahu agrees to join Trump’s Boar...,https://nypost.com/2026/01/21/world-news/israe...
163,36,2026-01-21-10-30-00 +0000,wsj,Google searches for “No Buy January” have hit ...,https://www.wsj.com/economy/consumers/no-buy-j...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
